In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from mesostat.metric.dim3d.npeet_pid import pid_barrett
from mesostat.metric.dim3d.pid_gaussian import pid_barrett_gaussian, pid_kayince_gaussian

%load_ext autoreload
%autoreload 2

In [ ]:
def gen_discr_rand(n):
    x = np.random.randint(0, 2, n)
    y = np.random.randint(0, 2, n)
    z = np.random.randint(0, 2, n)
    return x,y,z

def gen_discr_syn(n):
    x = np.random.randint(0, 2, n)
    y = np.random.randint(0, 2, n)
    z = np.logical_xor(x, y)
    return x,y,z

def gen_discr_red(n):
    x = np.random.randint(0, 2, n)
    y = x.copy()
    z = y.copy()
    return x,y,z

def gen_discr_unq(n):
    x = np.random.randint(0, 2, n)
    y = np.random.randint(0, 2, n)
    z = x.copy()
    return x,y,z

def gen_cont_rand(n):
    x = np.random.normal(0, 1, n)
    y = np.random.normal(0, 1, n)
    z = np.random.normal(0, 1, n)
    return x,y,z

# Continuous xor
def gen_cont_syn(n):
    x = np.random.normal(0, 1, n)
    y = np.random.normal(0, 1, n)
    z = np.abs(np.random.normal(0, 1, n))
    z *= np.sign(x) * np.sign(y)
    return x,y,z

def gen_cont_red(n):
    x = np.random.normal(0, 1, n)
    y = x.copy()
    z = x.copy()
    return x,y,z

def gen_cont_unq(n):
    x = np.random.normal(0, 1, n)
    y = np.random.normal(0, 1, n)
    z = x.copy()
    return x,y,z

def gen_cont_sum(n):
    x = np.random.normal(0, 1, n)
    y = np.random.normal(0, 1, n)
    z = x + y
    return x,y,z

## MMI numerical tests 

In [ ]:
n = 10000
nTest = 1000

In [ ]:
shuffleLst = [pid_barrett(*gen_cont_rand(n)) for iTest in range(nTest)]

In [ ]:
synLst = [pid_barrett(*gen_cont_syn(n)) for iTest in range(nTest)]

In [ ]:
def lst2df(lst, kind):
    atomTypes = ['U1', 'U2', 'red', 'syn']
    dfRez = pd.DataFrame()

    for l in lst:
        for iAtom, atom in enumerate(l):
            dfRez = dfRez.append(pd.DataFrame({'type': atomTypes[iAtom], 'val': atom, 'kind': kind}, index=[0]))
            
    return dfRez

In [ ]:
dfRand = lst2df(shuffleLst, 'rand')
dfData = lst2df(synLst, 'data')

In [ ]:
rezDF = dfData.copy()
rezDF = rezDF.append(dfRand)

In [ ]:
fig, ax = plt.subplots()
sns.violinplot(ax=ax, data=rezDF, x='type', y='val', hue='kind')
ax.set_yscale('log')

In [ ]:
print('Cont_rand', pid_barrett(*gen_cont_rand(n)))
print('Cont_red', pid_barrett(*gen_cont_red(n)))
print('Cont_syn', pid_barrett(*gen_cont_syn(n)))
print('Cont_sum', pid_barrett(*gen_cont_sum(n)))

# Gaussian Tests - Manual

In [ ]:
def mmi(Ix_y, Ix_z, Ix_yz):
    red = min(Ix_y, Ix_z)
    u1 = Ix_y - red
    u2 = Ix_z - red
    syn = Ix_yz - u1 - u2 - red
    return u1, u2, red, syn

def info_noisy_gau(alpha):
    k = 1/np.sqrt(alpha**2 + 2*(1-alpha)**2)
    a = k*(1-alpha)
    c = k*(1-alpha)
    b = 0
    detSig = 2*a*b*c - a**2 - b**2 - c**2 + 1
    Ix_y = np.log(1/(1-a**2))/2
    Ix_z = np.log(1/(1-c**2))/2
    Ix_yz = np.log((1-b**2)/detSig)/2
    dI = np.log((1-a**2)*(1-b**2)*(1-c**2)/detSig)/2
    return Ix_y, Ix_z, Ix_yz, dI

alphaLst = np.linspace(0.01, 1, 100)
rezLst = [info_noisy_gau(alpha) for alpha in alphaLst]
rezArr = np.array(rezLst).T
mmiLst = [mmi(*rez[:3]) for rez in rezLst]
mmiArr = np.array(mmiLst).T

namesMI = ['Ix_y', 'Ix_z', 'Ix_yz', 'dI']
namesPID = ['u1', 'u2', 'red', 'syn']

fig, ax = plt.subplots(ncols=2, figsize=(8,4))
for key, vals in zip(namesMI, rezArr):
    ax[0].semilogy(alphaLst, vals, label=key)
ax[0].legend()

for key, vals in zip(namesPID, mmiArr):
    ax[1].semilogy(alphaLst, vals, label=key)
ax[1].legend()

plt.show()

# Gaussian Tests - Impl

In [ ]:
noisy_1D = lambda x,alpha: (1-alpha)*x + alpha*np.random.normal(0,1,x.shape)

def noisy_3D(x,y,z,alpha):
    return [noisy_1D(x, 0.01), noisy_1D(y, 0.01), noisy_1D(z, alpha)]
#     return [noisy_1D(x, alpha), noisy_1D(y, alpha), noisy_1D(z, alpha)]

In [ ]:
n=1000

pidFuncDict = {'numMMI': pid_barrett, 'MMI': pid_barrett_gaussian, 'DEP': pid_kayince_gaussian}
contModelFuncDict = {
    'rand': gen_cont_rand,
    'unq': gen_cont_unq,
    'syn': gen_discr_syn,
    'red': gen_cont_red,
    'sum': gen_cont_sum
}

alphaLst = np.linspace(0.01, 1, 100)
fig, ax = plt.subplots(nrows=len(pidFuncDict), ncols=len(contModelFuncDict), 
                       figsize=(4*len(contModelFuncDict), 4*len(pidFuncDict)))

for iPIDFunc, (pidFuncName, pidFunc) in enumerate(pidFuncDict.items()):
    ax[iPIDFunc,0].set_ylabel(pidFuncName)
    
    for iModel, (modelName, modelFunc) in enumerate(contModelFuncDict.items()):    
        ax[0, iModel].set_title(modelName)

        rezLst = [pidFunc(*noisy_3D(*modelFunc(n), alpha)) for alpha in alphaLst]
        rezArr = np.clip(np.array(rezLst).T, 1.0e-7, None)

        namesPID = ['u1', 'u2', 'red', 'syn']

        for key, vals in zip(namesPID, rezArr):
            ax[iPIDFunc, iModel].semilogy(alphaLst, vals, label=key)
        ax[iPIDFunc, iModel].legend()
        ax[iPIDFunc, iModel].set_ylim(None, 10)

plt.show()

## PCorr vs Syn

In [ ]:
from mesostat.metric.dim3d.partialcorr import partial_corr
from mesostat.utils.decorators import redirect_stdout
from idtxl.bivariate_pid import BivariatePID
from idtxl.data import Data

def bernoulli(n, p):
    return (np.random.uniform(0, 1, n) < p).astype(int)

def _add_discr_noise(n, x, y, z, pX=0.5, pY=0.5, pZ=0.5):
    aX = bernoulli(n, pX)
    aY = bernoulli(n, pY)
    aZ = bernoulli(n, pZ)
    xNew = (1 - aX) * x + aX * bernoulli(n, 0.5)
    yNew = (1 - aY) * y + aY * bernoulli(n, 0.5)
    zNew = (1 - aZ) * z + aZ * bernoulli(n, 0.5)
    return xNew, yNew, zNew

@redirect_stdout
def pid_broja(x,y,z):
    dataPS = np.array([x,y,z], dtype=int)
    
    settings = {'pid_estimator': 'TartuPID', 'lags_pid': [0, 0]}

    dataIDTxl = Data(dataPS, dim_order='ps', normalise=False)
    pid = BivariatePID()
    rez = pid.analyse_single_target(settings=settings, data=dataIDTxl, target=2, sources=[0, 1])
    rezTrg = rez.get_single_target(2)

    # Getting rid of negative and very low positive PID's.
    # Statistical tests behave unexplectedly - perhaps low values contaminated by roundoff errors?
    return {k : np.clip(rezTrg[k], 1.0E-6, None) for k in ['unq_s1', 'unq_s2', 'shd_s1_s2', 'syn_s1_s2']}

## Version 1: PCorr vs BROJA Red

In [ ]:
n = 1000
alphaLst = np.linspace(0, 1, 100)

pCorrLst = []
redLst = []

for alpha in alphaLst:
#     print(alpha)
    x,y,z = _add_discr_noise(n, *gen_discr_red(n), 0,0,alpha)
    
    pCorrLst += [partial_corr(x,z,[y])]
    redLst += [pid_broja(x,y,z)['shd_s1_s2']]
    
fig, ax = plt.subplots(figsize=(4,4))
sns.regplot(redLst, pCorrLst, ax=ax)

# 
# ax.plot(redLst, pCorrLst, '.')
ax.set_title('Model: Z=X=Y (+noise)')
ax.set_xlabel('Redundancy')
ax.set_ylabel('Partial Correlation')
ax.set_xlim(-0.05, 1.05)
plt.show()

## Version 2: PCorr vs BROJA Unq

In [ ]:
n = 1000
alphaLst = np.linspace(0, 1, 100)

pCorrLst = []
unqLst = []

for alpha in alphaLst:
#     print(alpha)
    x,y,z = _add_discr_noise(n, *gen_discr_unq(n), 0,0, alpha)
    
    pCorrLst += [partial_corr(x,z,[y])]
    unqLst += [pid_broja(x,y,z)['unq_s1']]
    
fig, ax = plt.subplots(figsize=(4,4))
sns.regplot(unqLst, pCorrLst, ax=ax)

# 
ax.set_title('Model: Z=X (+noise)')
ax.set_xlabel('Unique Information')
ax.set_ylabel('Partial Correlation')
ax.set_xlim(-0.05, 1.05)
plt.show()

## Version 3: PCorr vs BROJA Syn (XOR)

In [ ]:
n = 1000
alphaLst = np.linspace(0, 1, 100)

pCorrLst = []
synLst = []

for alpha in alphaLst:
#     print(alpha)
    x,y,z = _add_discr_noise(n, *gen_discr_syn(n), 0,0, alpha)
    
    pCorrLst += [partial_corr(x,z,[y])]
    synLst += [pid_broja(x,y,z)['syn_s1_s2']]
    
fig, ax = plt.subplots(figsize=(4,4))
sns.regplot(synLst, pCorrLst, ax=ax)

# 
ax.set_title('Model: Z=X XOR Y (+noise)')
ax.set_xlabel('Synergistic Information')
ax.set_ylabel('Partial Correlation')
ax.set_xlim(-0.05, 1.05)
plt.show()

## Version 4: PCorr vs MMI PID Syn (X+Y)

In [ ]:
n = 1000
alphaLst = np.linspace(0, 1, 100)

pCorrLst = []
synLst = []

for alpha in alphaLst:
#     print(alpha)
    x,y,z = noisy_3D(*gen_cont_sum(n), alpha)
    
    pCorrLst += [partial_corr(x,z,[y])]
    synLst += [pid_barrett_gaussian(x,y,z)[3]]
    
fig, ax = plt.subplots(figsize=(4,4))
sns.regplot(synLst, pCorrLst, ax=ax)

# 
ax.set_title('Model: Z=X+Y (+noise)')
ax.set_xlabel('Synergistic Information')
ax.set_ylabel('Partial Correlation')
ax.set_xlim(-0.1, None)
ax.set_ylim(-0.05, 1.05)
plt.show()